# DistilBERT Sentiment Classification (HuggingFace) and Deployment

<a href="https://colab.research.google.com/github/VertaAI/modeldb/blob/master/client/workflows/demos/distilbert-sentiment-classification.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

This notebook walks through creating a few DistilBERT and BERT sentiment classification models, logging them to the Verta platform, and deploying them.

---

# Setup

First, we'll set up some environment variables so that the client can communicate with the Verta platform.

Change these to the values in your web app profile!

In [1]:
import os

os.environ['VERTA_HOST'] = "https://CHANGEME.verta.ai/"
os.environ['VERTA_EMAIL'] = "CHANGEME@CHANGEME.CHANGEME"
os.environ['VERTA_DEV_KEY'] = "CHANGEME"

Then we'll make sure we have the Python libraries we need:
- Verta for logging/deployment
- PyTorch + HuggingFace's `transformers` for the models themselves.

In [2]:
!python -m pip install verta torch==1.7.0 transformers

---

# Create a model

A model has to exist before we can log and deploy it, so we will instantiate one here in our notebook.

This model class will be an extensible wrapper over a [pre-trained HuggingFace classifier](https://huggingface.co/transformers/index.html).
- `predict()` is called when the model serves predictions.
- `describe()` is used to populate the deployed model's details in the summary page on the web app.
- `example()` is used to supply a sample input for the model playground on the web app.

In [3]:
from transformers import (
    pipeline,
    AutoModelForSequenceClassification,
    AutoTokenizer,
)

In [4]:
class _ModelBase:
    MODEL = None

    def __init__(self):
        self.model = pipeline(
            task="sentiment-analysis",
            model=AutoModelForSequenceClassification.from_pretrained(self.MODEL),
            tokenizer=AutoTokenizer.from_pretrained(self.MODEL),
        )

    def predict(self, texts):
        return self.model(texts)
    
    def describe(self):
        return {
            'method': 'predict',
            'args': 'texts',
            'returns': 'sentiments',
            'description': """
                Labels each sentiment as either positive or negative, along with a score for that classification.
            """,
            'input_description': """
                list[str] texts: Statements to classify.
            """,
            'output_description': """
                list[dict] sentiments: Label and score for each statement.
                    'label' is "POSITIVE" or "NEGATIVE".
                    'score' is a float between 0 and 1.
            """
        }

    def example(self):
        return [
            "I like you",
            "I don't like this film",
        ]


class DistilBERT(_ModelBase):
    MODEL = "distilbert-base-uncased-finetuned-sst-2-english"

    def predict(self, texts):
        sentiments = super(DistilBERT, self).predict(texts)

        return sentiments

In [5]:
distilbert = DistilBERT()

texts = distilbert.example()
print(texts)
print(distilbert.predict(texts))

# Log model to Verta

Now that the model is in a good shape, we can log it to the Verta platform. First, we initialize the client to connect with the system.

In [6]:
from verta import Client

client = Client()

We'll create a [project](https://verta.readthedocs.io/en/master/_autogen/verta.tracking.entities.Project.html) for our text classification models,  
an [experiment](https://verta.readthedocs.io/en/master/_autogen/verta.tracking.entities.Experiment.html) within it for models based on the DistilBERT architecture,  
and an [experiment run](https://verta.readthedocs.io/en/master/_autogen/verta.tracking.entities.ExperimentRun.html) to associate our model with.

All of these can be viewed in the Verta web app once they are created.

In [7]:
proj = client.get_or_create_project(
    "Text Classification",
    desc="Models trained for textual sentiment classification.",
    tags=["NLP", "Classification", "Q4"],
    attrs={'team': "Verta"},
)

expt = client.get_or_create_experiment("DistilBERT", tags=["Neural Net"])

run = client.create_experiment_run("First DistilBERT", tags=["DistilBERT", "English"])

run

Then the [model itself will be uploaded](https://verta.readthedocs.io/en/master/_autogen/verta.tracking.entities.ExperimentRun.html#verta.tracking.entities.ExperimentRun.log_model), along with [a list of Python libraries](https://verta.readthedocs.io/en/master/_autogen/verta.tracking.entities.ExperimentRun.html#verta.tracking.entities.ExperimentRun.log_environment) that will be required to deploy it.

In [8]:
# from https://huggingface.co/distilbert-base-uncased-finetuned-sst-2-english
run.log_hyperparameters({
    'learning_rate': 1e-5,
    'batch_size': 32,
    'max_seq_length': 128,
    'num_train_epochs': 3,
})
run.log_metric('accuracy', .913)

run.log_model(distilbert, custom_modules=[])
run.log_requirements(["torch==1.7.0", "transformers"])

run

Once the model is uploaded to a run, it can be deployed through the web app or [via the client](https://docs.verta.ai/verta/deployment/guides/endpoint-update) as shown below, whereupon predictions can be made via a REST endpoint.

In [9]:
import uuid

endpoint = client.get_or_create_endpoint(path=str(uuid.uuid4()))
endpoint.update(run, wait=True)

endpoint

Predictions can also be made through the client!

In [10]:
deployed_model = endpoint.get_deployed_model()

print(texts)
print(deployed_model.predict(texts))

---

# Log additional models

One key aspect of the Verta platform is being able to organize your various trained models in a easy-to-navigate structure.  
We can log additional BERT models to our project—under a new experiment—and be able to reference and deploy them as well.

In [11]:
client.create_experiment("BERT", tags=["Neural Net"])

In [12]:
class BERT(_ModelBase):
    MODEL = "textattack/bert-base-uncased-imdb"

    def predict(self, texts):
        sentiments = super(BERT, self).predict(texts)

        # fix labels
        for sentiment in sentiments:
            if sentiment['label'] == "LABEL_0":
                sentiment['label'] = "NEGATIVE"
            else:  # "LABEL_1"
                sentiment['label'] = "POSITIVE"

        return sentiments
    
bert = BERT()
run = client.create_experiment_run(
    "First BERT",
    tags=["BERT", "English"],
)

# from https://huggingface.co/textattack/bert-base-uncased-imdb
run.log_hyperparameters({
    'learning_rate': 2e-5,
    'batch_size': 16,
    'max_seq_length': 128,
    'num_train_epochs': 5,
})
run.log_metric('accuracy', .89088)

run.log_model(bert, custom_modules=[])
run.log_requirements(["torch==1.7.0", "transformers"])

run

In [13]:
class GermanBERT(_ModelBase):
    MODEL = "oliverguhr/german-sentiment-bert"

    def predict(self, texts):
        sentiments = super(GermanBERT, self).predict(texts)
        
        # fix labels
        for sentiment in sentiments:
            sentiment['label'] = sentiment['label'].upper()

        return sentiments

german_bert = GermanBERT()
run = client.create_experiment_run(
    "German",
    tags=["BERT", "German"],
)

# from http://www.lrec-conf.org/proceedings/lrec2020/pdf/2020.lrec-1.202.pdf
run.log_hyperparameters({
    'learning_rate': 2e-5,
    'batch_size': 32,
    'max_seq_length': 256,
    'num_train_epochs': 3,
})
run.log_metric('f1', .9639)

run.log_model(german_bert, custom_modules=[])
run.log_requirements(["torch==1.7.0", "transformers"])

run

In [14]:
class MultilingualBERT(_ModelBase):
    MODEL = "nlptown/bert-base-multilingual-uncased-sentiment"

    def __init__(self):
        super(MultilingualBERT, self).__init__()
        self.model.return_all_scores = True  # this model has 5 categories, and we'll need to make it 2


    def predict(self, texts):
        texts_scores = super(MultilingualBERT, self).predict(texts)

        # fix labels and scores
        sentiments = []
        for scores in texts_scores:
            # aggregate negative and positive scores
            negative_scores = filter(lambda score: score['label'] in {"1 star", "2 stars", "3 stars"}, scores)
            positive_scores = filter(lambda score: score['label'] in {"4 stars", "5 stars"}, scores)
            negative_score = sum(score['score'] for score in negative_scores)
            positive_score = sum(score['score'] for score in positive_scores)

            # select greater value as sentiment
            if positive_score > negative_score:
                label, score = "POSITIVE", positive_score
            else:
                label, score = "NEGATIVE", negative_score
            sentiments.append({'label': label, 'score': score})

        return sentiments

multilingual_bert = MultilingualBERT()
run = client.create_experiment_run(
    "Multilingual",
    tags=["BERT", "English", "German"],
)

# from https://huggingface.co/nlptown/bert-base-multilingual-uncased-sentiment
run.log_hyperparameters({  # example values; true hyperparameters not provided by model contributors
    'learning_rate': 2e-5,
    'batch_size': 16,
    'max_seq_length': 128,
    'num_train_epochs': 3,
})
run.log_metric('accuracy', .95)

run.log_model(multilingual_bert, custom_modules=[])
run.log_requirements(["torch==1.7.0", "transformers"])

run

---

# ...and beyond!

Now we have a handful of models logged and able to serve predictions!

Feel free to use these as a starting template for your own models, and take a look at [Verta's example notebooks](https://github.com/VertaAI/modeldb/tree/master/client/workflows) for more possibilities!

---